In [15]:
import pandas as pd
import torch
import torchvision
import os
import glob
import numpy as np

6,915 normal cells

In [36]:
# read in regulon activity matrix
dta_nn = pd.read_pickle("/n/holyscratch01/stephenson_lab/Users/taylerli/SCENIC/data/BM_normal_output/BM_normal_aucell_values.pkl.gz")
# read in correct annotations
anno_names = os.path.join("/n/holyscratch01/stephenson_lab/Users/taylerli/SCENIC/data", "GSM*_BM*.anno.txt.gz")
anno_names = glob.glob(anno_names)
anno_mat = pd.read_csv(anno_names[0], sep='\t', header=0, index_col=0)
for i in range(1, len(anno_names)):
    anno_mat = pd.concat([anno_mat, pd.read_csv(anno_names[i], sep='\t', header=0, index_col=0)], axis = 0)
# join annotation to regulon activity matrix
dta_nn = pd.merge(dta_nn, anno_mat['CellType'], on = 'Cell', how = 'left')

# partition out training set, validation set, and test set using stratified sampling
np.random.seed(292876)
train_set = dta_nn.groupby('CellType', group_keys=False).apply(lambda x: x.sample(frac=0.7))
train_set = train_set.sample(frac = 1)
validation_set = dta_nn.drop(train_set.index).groupby('CellType', group_keys=False).apply(lambda x: x.sample(frac=1/3))
test_set = dta_nn.drop(train_set.index).drop(validation_set.index)

# separate X train from y train
X_train = train_set.drop('CellType', axis = 1)
y_train = train_set['CellType']

# get X validation and y validation
X_validation = validation_set.drop('CellType', axis = 1)
y_validation = validation_set['CellType']

# get X test and y test
X_test = test_set.drop('CellType', axis = 1)
y_test = test_set['CellType']

In [41]:
train_set.to_pickle("train_set.pkl.gz")
validation_set.to_pickle("validation_set.pkl.gz")
test_set.to_pickle("test_set.pkl.gz")

20,148 healthy and cancer cells

In [16]:
# read in regulon activity matrix
dta_nn = pd.read_pickle("/n/holyscratch01/stephenson_lab/Users/taylerli/SCENIC/data/BM_AML_normal_output/BM_AML_normal_aucell_values.pkl.gz")
# read in correct annotations
anno_names = os.path.join("/n/holyscratch01/stephenson_lab/Users/taylerli/SCENIC/data", "GSM*_*.anno.txt.gz")
anno_names = glob.glob(anno_names)
anno_mat = pd.read_csv(anno_names[0], sep='\t', header=0, index_col=0)
for i in range(1, len(anno_names)):
    anno_mat = pd.concat([anno_mat, pd.read_csv(anno_names[i], sep='\t', header=0, index_col=0)], axis = 0)
# join annotation to regulon activity matrix
dta_nn = pd.merge(dta_nn, anno_mat['CellType'], on = 'Cell', how = 'left')
dta_nn['HC'] = np.where(dta_nn.CellType.str[-4:] == 'like', 'Cancer', 'Healthy')
# drop unuseful regulons, according to LASSO
dta_nn = dta_nn.drop(dta_nn.columns[[11, 45, 63, 67, 77, 86, 108, 134, 158, 218, 219, 241, 265, 269, 297, 303, 312]], axis = 1)

# partition out training set, validation set, and test set using stratified sampling
np.random.seed(292876)
train_set = dta_nn.groupby('CellType', group_keys=False).apply(lambda x: x.sample(frac=0.7))
train_set = train_set.sample(frac = 1)
validation_set = dta_nn.drop(train_set.index).groupby('CellType', group_keys=False).apply(lambda x: x.sample(frac=1/3))
test_set = dta_nn.drop(train_set.index).drop(validation_set.index)

# separate X train from y train
X_train = train_set.drop(['CellType', 'HC'], axis = 1)
y_train = train_set['HC']

# get X validation and y validation
X_validation = validation_set.drop(['CellType', 'HC'], axis = 1)
y_validation = validation_set['HC']

# get X test and y test
X_test = test_set.drop(['CellType', 'HC'], axis = 1)
y_test = test_set['HC']

6,905 cancer cells

In [40]:
# read in regulon activity matrix
dta_nn = pd.read_pickle("/n/holyscratch01/stephenson_lab/Users/taylerli/SCENIC/data/BM_AML_normal_output/BM_AML_normal_aucell_values.pkl.gz")
# read in correct annotations
anno_names = os.path.join("/n/holyscratch01/stephenson_lab/Users/taylerli/SCENIC/data", "GSM*_*.anno.txt.gz")
anno_names = glob.glob(anno_names)
anno_mat = pd.read_csv(anno_names[0], sep='\t', header=0, index_col=0)
for i in range(1, len(anno_names)):
    anno_mat = pd.concat([anno_mat, pd.read_csv(anno_names[i], sep='\t', header=0, index_col=0)], axis = 0)
# join annotation to regulon activity matrix
dta_nn = pd.merge(dta_nn, anno_mat['CellType'], on = 'Cell', how = 'left')
dta_nn['HC'] = np.where(dta_nn.CellType.str[-4:] == 'like', 'Cancer', 'Healthy')


dta_nn1 = dta_nn[dta_nn['HC'] == 'Cancer']
train_set = dta_nn1.groupby('CellType', group_keys=False).apply(lambda x: x.sample(frac=0.7))
train_set = train_set.sample(frac = 1)
validation_set = dta_nn1.drop(train_set.index).groupby('CellType', group_keys=False).apply(lambda x: x.sample(frac=1/3))
test_set = dta_nn1.drop(train_set.index).drop(validation_set.index)

# separate X train from y train
X_train = train_set.drop(['CellType', 'HC'], axis = 1)
y_train = train_set['CellType']

# get X validation and y validation
X_validation = validation_set.drop(['CellType', 'HC'], axis = 1)
y_validation = validation_set['CellType']

# get X test and y test
X_test = test_set.drop(['CellType', 'HC'], axis = 1)
y_test = test_set['CellType']

Regulon composition lookup

In [39]:
def print0(x):
    lis = [ele for ele in x]
    for l in lis:
        print(l)

c = pd.read_pickle("/n/holyscratch01/stephenson_lab/Users/taylerli/SCENIC/data/BM_AML_normal_output/BM_AML_normal_regulons.p")
print0(c[[i for i in range(len(c)) if c[i].name == 'TAGLN2(+)'][0]].gene2weight.keys())

ZIC2
PSMD13
MGAT1
PDLIM2
PRKCD
EPN1
NCF4
KIAA0232
LASP1
PRKG1
SV2C
ARPC4
RHOG
IL2RA
GPSM3
CCND3
ACTG1
ARHGAP1
PGS1
CARD9
RALB
RAB31
SSTR2
SLC9A3R1
SLC16A3
CFP
RTN4
HLA-DPB2
ARPC1B
CD163L1
TEX264
RGS14
RAB27B
HLA-C
TWF2
XAF1
CAP1
MPG
LSP1
HEXIM1
CSK
CAMK1D
ANTXR2
DPEP2
PFN1
S100A10
SLC9A3R2
HPCAL1
C2orf88
NCALD
TNNI2
RAB11FIP1
CTSB
AKNA
ARF1
ADAM15
LRRK1
HLA-DQA1
FILIP1L
KIAA0513
VPS4B
SNAI3
DLL4
DGKG
NAAA
FGD2
AP1G2
URGCP
HLA-DMB
PLCB2
LRCH4
TNFAIP8L2
MTMR14
CD36
ITGA2B
CDON
GNAL
CDC42SE1
SLAIN1
UCP2
TIAF1
ERP29
LTBP1
TMBIM1
PEA15
CORO1B
ARHGAP27
ERCC1
CD52
RND1
RELT
LPXN
RPS6KA1
PPFIBP2
GAS7
ZMIZ1
CORO1A
OSBPL3
PLEKHM1
EMP1
ARHGAP6
TRIM16
HLA-DRA
PLEK
GSDMD
DPYSL3
RAB1B
PILRB
CYTH4
ARL2BP
MSN
ITGB7
RALY
C1QTNF2
ITGA11
GNB2
C17orf62
EMX2
TNIP1
PRR5L
ALOX5
NFE2L1
ATP10D
CAPN1
ADRB2
CALCRL
DLGAP4
PLCD4
CTDSPL


Neural Network class object

In [2]:
class NeuralNetwork:
    
    def __init__(self, hidden = [50], act_func = ['sigmoid'], epochs = 10, lr = 0.1, iter_size = 4):
        if len(hidden) != len(act_func):
            raise ValueError('The number of hidden layers and the number of activation functions must be the same.')
        self.__hidden = hidden
        self.__act_func = act_func
        self.__epochs = epochs
        self.__lr = lr
        self.__iter_size = iter_size
        self.X_train = None
        self.y_train = None
        self.params = []
        self.history_accuracy = []
        self.history_loss = []
    
    def modify(self, hidden = None, act_func = None, epochs = None, lr = None, iter_size = None):
        if hidden is not None:
            if act_func is not None:
                if len(hidden) != len(act_func):
                    raise ValueError('The number of hidden layers and the number of activation functions must be the same.')
                else:
                    self.__hidden = hidden
                    self.__act_func = act_func
            else:
                if len(hidden) != len(self.__act_func):
                    raise ValueError('The number of hidden layers and the number of activation functions must be the same.')
                else:
                    self.__hidden = hidden
        elif act_func is not None:
            if len(self.__hidden) != len(act_func):
                raise ValueError('The number of hidden layers and the number of activation functions must be the same.')
            else:
                self.__act_func = act_func
        elif epochs is not None:
            self.__epochs = epochs
        elif lr is not None:
            self.__lr = lr
        elif iter_size is not None:
            self.__iter_size = iter_size
        else:
            raise ValueError('Must make at least one modification!')
            
    def refresh(self):
        self.params = []
        self.history_accuracy = []
        self.history_loss = []
    
    def fit(self, X, y, scale_X = True):
        # getting training X and y
        y = pd.get_dummies(y)    # one-hot encoding training y
        self.y_train = torch.from_numpy(y.values).float()
        if scale_X:
            X = (X-X.min())/(X.max()-X.min())
        self.X_train = torch.from_numpy(X.values).float()
        
        # initializing weights and bias
        for i in range(len(self.__hidden)):
            if i == 0:
                W = torch.nn.Parameter(0.01 * torch.randn(size = (self.X_train.shape[1], self.__hidden[i])))
            else:
                W = torch.nn.Parameter(0.01 * torch.randn(size = (self.__hidden[i-1], self.__hidden[i])))
            b = torch.nn.Parameter(torch.zeros(self.__hidden[i]))
            self.params.append(W)
            self.params.append(b)
        W_final = torch.nn.Parameter(0.01 * torch.randn(size = (self.__hidden[len(self.__hidden)-1], self.y_train.shape[1])))
        b_final = torch.nn.Parameter(torch.zeros(self.y_train.shape[1]))
        self.params.append(W_final)
        self.params.append(b_final)
        
    def __ReLU__(self, X):
        return torch.clamp(X, min = 0)

    def __leakyReLU__(self, X):
        return torch.where(X >= 0, X, 0.1*X)

    def __sigmoid__(self, X):
        return 1 / (1 + torch.exp(-X))

    def __tanh__(self, X):
        return torch.tanh(X)

    def __ELU__(self, X):
        return torch.where(X >= 0, X, 0.1*(torch.exp(X) - 1))

    def __softmax__(self, X):
        e = torch.exp(X)
        exp_sum = torch.sum(e, dim = 1, keepdim = True)
        soft = e/exp_sum
        return soft
        
    def __forwardPass__(self, X):
        for j in range(len(self.params) // 2):
            W = self.params[j*2]
            b = self.params[j*2 + 1]
            if j < len(self.params) // 2 - 1:
                func = eval('self.__' + self.__act_func[j] + '__')
                if j == 0:
                    H = func(X@W + b)
                else:
                    H = func(H@W + b)
            else:
                O = self.__softmax__(H@W + b)
        return O
    
    def __accuracy__(self, y_hat, y):
        with torch.no_grad():
            y_pred = y_hat.argmax(axis=1)
            y_true = y.argmax(axis = 1)
            correct = y_pred == y_true
        return correct.sum() / correct.numel()
        
    def __crossEntropy__(self, y_hat, y):
        ln_yhat = torch.log(y_hat)
        cr_en = -torch.sum(y * ln_yhat, dim = 1)
        return cr_en.mean()
    
    def __update__(self):
        with torch.no_grad():
            for param in self.params:
                param -= self.__lr * param.grad
                param.grad.zero_()
        return
        
    def train(self):
        if self.X_train == None:
            raise ValueError('Must need a training set to start training.')
        for _ in range(self.__epochs):
            accuracy = 0.0
            loss = 0.0
            n_trials = 0
            for i in range((self.X_train.shape[0] - 1) // self.__iter_size + 1):
                n_trials += 1
                X_iter = self.X_train[i*self.__iter_size:(i+1)*self.__iter_size, :]
                y_iter = self.y_train[i*self.__iter_size:(i+1)*self.__iter_size, :]
                y_hat = self.__forwardPass__(X_iter)
                accuracy += self.__accuracy__(y_hat, y_iter)
                l = self.__crossEntropy__(y_hat, y_iter)
                loss += l
                l.backward()
                self.__update__()
            self.history_accuracy.append(accuracy.item() / n_trials)
            self.history_loss.append(loss.item())
    
    def get_accuracy(self, X = None, y = None, scale_X = True):
        if X is not None and y is not None:
            y = pd.get_dummies(y)
            if scale_X:
                X = (X-X.min())/(X.max()-X.min())
            y_test = torch.from_numpy(y.values).float()
            X_test = torch.from_numpy(X.values).float()
            return self.__accuracy__(self.__forwardPass__(X_test), y_test).item()
        else:
            return self.__accuracy__(self.__forwardPass__(self.X_train), self.y_train).item()

In [ ]:
for ele in ['sigmoid', 'tanh', 'ReLU', 'leakyReLU', 'ELU']:
    NN = NeuralNetwork([50], [ele], epochs = 40, lr = 0.05)
    NN.fit(X_train, y_train)
    NN.train()
    print('The training set accuracy is ' + str(NN.get_accuracy()))
    print('The validation set accuracy is ' + str(NN.get_accuracy(X_validation, y_validation)))

The training set accuracy is 0.9163222908973694
The validation set accuracy is 0.8075253367424011
The training set accuracy is 0.9661157131195068
The validation set accuracy is 0.7539797425270081
The training set accuracy is 0.016322314739227295
The validation set accuracy is 0.01591895893216133
The training set accuracy is 0.016322314739227295
The validation set accuracy is 0.01591895893216133


In [6]:
for ele in ['sigmoid', 'tanh', 'ReLU', 'leakyReLU', 'ELU']:
    NN = NeuralNetwork([50], [ele], epochs = 40, lr = 0.05)
    NN.fit(X_train, y_train, scale_X=False)
    NN.train()
    print('The training set accuracy is ' + str(NN.get_accuracy()))
    print('The validation set accuracy is ' + str(NN.get_accuracy(X_validation, y_validation, scale_X=False)))

The training set accuracy is 0.8444215059280396
The validation set accuracy is 0.7973950505256653
The training set accuracy is 0.9212809801101685
The validation set accuracy is 0.8321273326873779
The training set accuracy is 0.9179751873016357
The validation set accuracy is 0.8263386487960815
The training set accuracy is 0.9130165576934814
The validation set accuracy is 0.8219971060752869
The training set accuracy is 0.921074390411377
The validation set accuracy is 0.8306801915168762


In [5]:
NN = NeuralNetwork([100, 50], ['tanh', 'ELU'], epochs = 40, lr = 0.05)
NN.fit(X_train, y_train, scale_X=False)
NN.train()
print('The training set accuracy is ' + str(NN.get_accuracy()))
print('The validation set accuracy is ' + str(NN.get_accuracy(X_validation, y_validation, scale_X=False)))

The training set accuracy is 0.8704545497894287
The validation set accuracy is 0.7814761400222778


In [6]:
NN = NeuralNetwork([100, 50], ['tanh', 'sigmoid'], epochs = 40, lr = 0.05)
NN.fit(X_train, y_train, scale_X=False)
NN.train()
print('The training set accuracy is ' + str(NN.get_accuracy()))
print('The validation set accuracy is ' + str(NN.get_accuracy(X_validation, y_validation, scale_X=False)))

The training set accuracy is 0.8535124063491821
The validation set accuracy is 0.80173659324646


In [7]:
NN = NeuralNetwork([100, 50], ['ELU', 'tanh'], epochs = 40, lr = 0.05)
NN.fit(X_train, y_train, scale_X=False)
NN.train()
print('The training set accuracy is ' + str(NN.get_accuracy()))
print('The validation set accuracy is ' + str(NN.get_accuracy(X_validation, y_validation, scale_X=False)))

The training set accuracy is 0.8962810039520264
The validation set accuracy is 0.784370481967926


In [14]:
NN = NeuralNetwork([100, 50], ['ELU', 'ELU'], epochs = 40, lr = 0.05)
NN.fit(X_train, y_train, scale_X=False)
NN.train()
print('The training set accuracy is ' + str(NN.get_accuracy()))
print('The validation set accuracy is ' + str(NN.get_accuracy(X_validation, y_validation, scale_X=False)))

The training set accuracy is 0.016322314739227295
The validation set accuracy is 0.01591895893216133


In [15]:
for ele in ['sigmoid', 'tanh', 'ReLU', 'leakyReLU', 'ELU']:
    NN = NeuralNetwork([50], [ele], epochs = 50, lr = 0.05)
    NN.fit(X_train, y_train, scale_X=False)
    NN.train()
    print('The training set accuracy is ' + str(NN.get_accuracy()))
    print('The validation set accuracy is ' + str(NN.get_accuracy(X_validation, y_validation, scale_X=False)))

The training set accuracy is 0.863223135471344
The validation set accuracy is 0.8075253367424011
The training set accuracy is 0.9357438087463379
The validation set accuracy is 0.8335745334625244
The training set accuracy is 0.9287189841270447
The validation set accuracy is 0.8191027641296387
The training set accuracy is 0.9334710836410522
The validation set accuracy is 0.8277857899665833
The training set accuracy is 0.9314049482345581
The validation set accuracy is 0.8306801915168762


In [16]:
NN = NeuralNetwork([100, 30], ['tanh', 'sigmoid'], epochs = 50, lr = 0.1)
NN.fit(X_train, y_train, scale_X=False)
NN.train()
print('The training set accuracy is ' + str(NN.get_accuracy()))
print('The validation set accuracy is ' + str(NN.get_accuracy(X_validation, y_validation, scale_X=False)))

The training set accuracy is 0.9084710478782654
The validation set accuracy is 0.8046309947967529


In [49]:
NN = NeuralNetwork([100, 100], ['sigmoid', 'sigmoid'], epochs = 100, lr = 0.1)
NN.fit(X_train, y_train, scale_X=False)
NN.train()
print('The training set accuracy is ' + str(NN.get_accuracy()))
print('The validation set accuracy is ' + str(NN.get_accuracy(X_validation, y_validation, scale_X=False)))
#print('The validation set accuracy is ' + str(NN.get_accuracy(X_test, y_test, scale_X=False)))

The training set accuracy is 0.7839850783348083
The validation set accuracy is 0.7366136312484741


In [48]:
NN = NeuralNetwork([100, 50], ['tanh', 'tanh'], epochs = 100, lr = 0.1)
NN.fit(X_train, y_train, scale_X=False)
NN.train()
print('The training set accuracy is ' + str(NN.get_accuracy()))
print('The validation set accuracy is ' + str(NN.get_accuracy(X_validation, y_validation, scale_X=False)))

The training set accuracy is 0.832402229309082
The validation set accuracy is 0.730824887752533


In [17]:
NN = NeuralNetwork([100, 50], ['tanh', 'sigmoid'], epochs = 50, lr = 0.1)
NN.fit(X_train, y_train, scale_X=False)
NN.train()
print('The training set accuracy is ' + str(NN.get_accuracy()))
print('The validation set accuracy is ' + str(NN.get_accuracy(X_validation, y_validation, scale_X=False)))

The training set accuracy is 0.910330593585968
The validation set accuracy is 0.8205499053001404


In [5]:
NN = NeuralNetwork([50, 30], ['tanh', 'sigmoid'], epochs = 100, lr = 0.1)
NN.fit(X_train, y_train, scale_X=False)
NN.train()
print('The training set accuracy is ' + str(NN.get_accuracy()))
print('The validation set accuracy is ' + str(NN.get_accuracy(X_validation, y_validation, scale_X=False)))

The training set accuracy is 0.9700413346290588
The validation set accuracy is 0.8321273326873779


In [6]:
NN = NeuralNetwork([50, 30], ['tanh', 'sigmoid'], epochs = 100, lr = 0.05)
NN.fit(X_train, y_train, scale_X=False)
NN.train()
print('The training set accuracy is ' + str(NN.get_accuracy()))
print('The validation set accuracy is ' + str(NN.get_accuracy(X_validation, y_validation, scale_X=False)))

The training set accuracy is 0.9510330557823181
The validation set accuracy is 0.8205499053001404


In [7]:
NN = NeuralNetwork([50, 50], ['tanh', 'sigmoid'], epochs = 100, lr = 0.05)
NN.fit(X_train, y_train, scale_X=False)
NN.train()
print('The training set accuracy is ' + str(NN.get_accuracy()))
print('The validation set accuracy is ' + str(NN.get_accuracy(X_validation, y_validation, scale_X=False)))

The training set accuracy is 0.9285123944282532
The validation set accuracy is 0.8118668794631958


In [42]:
for ele in ['sigmoid', 'tanh', 'ReLU', 'leakyReLU', 'ELU']:
    NN = NeuralNetwork([100], [ele], epochs = 100, lr = 0.05)
    NN.fit(X_train, y_train, scale_X=False)
    NN.train()
    print('The training set accuracy is ' + str(NN.get_accuracy()))
    print('The validation set accuracy is ' + str(NN.get_accuracy(X_validation, y_validation, scale_X=False)))

The training set accuracy is 0.81688392162323
The validation set accuracy is 0.7424023151397705
The training set accuracy is 0.8427477478981018
The validation set accuracy is 0.730824887752533
The training set accuracy is 0.18456445634365082
The validation set accuracy is 0.1837916076183319
The training set accuracy is 0.18456445634365082
The validation set accuracy is 0.1837916076183319
The training set accuracy is 0.18456445634365082
The validation set accuracy is 0.1837916076183319


In [43]:
NN = NeuralNetwork([200, 100], ['sigmoid', 'sigmoid'], epochs = 100, lr = 0.05)
NN.fit(X_train, y_train, scale_X=False)
NN.train()
print('The training set accuracy is ' + str(NN.get_accuracy()))
print('The validation set accuracy is ' + str(NN.get_accuracy(X_validation, y_validation, scale_X=False)))

The training set accuracy is 0.7502586245536804
The validation set accuracy is 0.7221418023109436


In [50]:
np.random.seed(292876)
NN = NeuralNetwork([150], ['sigmoid'], epochs = 100, lr = 0.05)
NN.fit(X_train, y_train, scale_X=False)
NN.train()
print('The training set accuracy is ' + str(NN.get_accuracy()))
print('The validation set accuracy is ' + str(NN.get_accuracy(X_validation, y_validation, scale_X=False)))

The training set accuracy is 0.81688392162323
The validation set accuracy is 0.7395079731941223


In [51]:
np.random.seed(292876)
NN = NeuralNetwork([100], ['sigmoid'], epochs = 100, lr = 0.05)
NN.fit(X_train, y_train, scale_X=False)
NN.train()
print('The training set accuracy is ' + str(NN.get_accuracy()))
print('The validation set accuracy is ' + str(NN.get_accuracy(X_validation, y_validation, scale_X=False)))
#print('The test set accuracy is ' + str(NN.get_accuracy(X_test, y_test, scale_X=False)))

The training set accuracy is 0.8181253671646118
The validation set accuracy is 0.7395079731941223


In [52]:
print('The test set accuracy is ' + str(NN.get_accuracy(X_test, y_test, scale_X=False)))

The test set accuracy is 0.7219406366348267


In [20]:
np.random.seed(292876)
NN = NeuralNetwork([100], ['ReLU'], epochs = 100, lr = 0.05)
NN.fit(X_train, y_train, scale_X=False)
NN.train()
print('The training set accuracy is ' + str(NN.get_accuracy()))
print('The validation set accuracy is ' + str(NN.get_accuracy(X_validation, y_validation, scale_X=False)))
print('The test set accuracy is ' + str(NN.get_accuracy(X_test, y_test, scale_X=False)))

The training set accuracy is 0.9930511116981506
The validation set accuracy is 0.9667987823486328
The test set accuracy is 0.9634963870048523
